# Transistor selection
## Limitations
* Transistor input capacitance Cin = Ciss = Cgs + Cgd
* Transistor output capacitance Cout = Coss = Cds + Cgd

## Output power calculations

$$Vout = VDD * \sqrt{D}$$

Where Vout = output voltage, VDD = supply voltage, D = duty cycle.

## Comparison
### Characteristics
IRLML2060TRPBF
* 60V, 1.2A
* SOT23-3 package can be hand soldered.
* Ciss = 64pF
* Coss = 13pF
* Gate Charge (Qg) (Max) @ Vgs : 0.67 nC @ 4.5 V
* tr = 3.8ns, tf = 2.8ns
* Needs protection circuitry to handle over voltage when load is short-circuited.

IRLML0100TRPbF
* 100V, 1.6A
* SOT23-3
* tr = 2.1ns, tf = 3.6ns 
* Ciss = 290pF
* Coss = 27pF
* Qg = 2.5nC

#### Performance
##### IRLML2060
* Gate current = 62mA (5V)
* RF-choke current = 346mA (12V)
* 50ohm load current = 272mA
Input power = 4.46W
Output power = 3.7W
Efficiency = 83%

##### IRLML0100
* Gate current = 137mA
* RF-choke current = 315mA
* 50ohm load current = 255mA
Input power = 4.47W
Output power = 3.25W
Efficiency = 73%

## Conclusion
The IRLML0100 can handle higher drain voltages, but simulations show that this part will break too when output is short circuited on an unprotected amplifier.  The IRLML0100 is less efficient, so there's little reason to choose this part over the IRLML2060.

# Part selection

## RF-choke
1A, SRF>100MHz, L>3µH, package 1210
* BRL3225T4R7M, BRL3225T3R3M, CBC3225T3R3MR
Will put two or three of these in series.

# Protection

Needed in case of open load or shorted load.
## MOSFET-overvoltage protection
* TVS has too much capacitance.  Varistor and TVS have a too big difference between the breakdown voltage (start of clamping action) and the maximum clamping voltage.  
* Varistors : The maximum clamping voltage is about 60% to 100% higher than the breakdown voltage.
* TVS-diode : maximum clamping voltage is about 50% higher than the breakdown voltage.  Breakdown voltage is 20% above rated voltage.
